# Laboratorio de regresión - 5

|                |   |
:----------------|---|
| **Nombre**     |Alan Yael Almaraz Parra|
| **Fecha**      |16/02/2026|
| **Expediente** |751171|

## Validación

Hemos estado usando `train_test_split` en nuestros modelos anteriores.

¿Por qué?

No, porque estamos aprendiendo el detrás y el como se utilizan las cosas sin la necesidad de usar librerias

Si la muestra es un subset de la población y queremos generalizar sobre la población, ¿no sería mejor utilizar todos los datos al entrenar un modelo?

El propósito de volver a muestrear dentro de nuestro dataset es tener una idea de qué tan buena podría ser la generalización de nuestro modelo. Imagina un dataset ya separado en dos mitades. Utilizas la primera mitad para entrenar el modelo y pruebas en la segunda mitad; la segunda mitad eran datos invisibles para el modelo al momento de entrenar. Esto nos lleva a tres escenario típicos:

1. Si el modelo hace buenas predicciones en la segunda mitad, significa que la primera mitad era "suficiente" para generalizar.
2. Si el modelo no hace buenas predicciones en la segunda mitad, pero sí en la primera mitad, podría ser que había información importante en la segunda mitad que debió haber sido tomada en cuenta al entrenar, o un problema de overfitting.
3. Si el modelo no hace buenas predicciones en la segunda mitad, y tampoco en la primera mitad, se tendrían que revisar los factores y/o el modelo seleccionado.

El caso ideal sería el 1, pero por estadística los errores y varianzas tienen como entrada el número de muestas, por lo que tenemos menos seguridad de nuestros resutados al usar menos muestras. Si vemos que el modelo generaliza bien podemos unir de nuevo el dataset y entrenar sobre el dataset completo.

En el caso 2 está el problema de que no podemos saber qué información es necesaria para el entrenamiento apropiado del modelo; esto nos lleva a pensar que debemos usar el dataset completo para entrenar, pero esto nos lleva al mismo problema de no saber si el modelo puede generalizar.

El problema sólo incrementa si se tienen hiperparámetros en el modelo (e.g. $\lambda$ en regularización).

## Leave-One-Out Cross Validation

Este método de validación es una colección de $n$ `train-test-split`. Teniendo un dataset de $n$ muestras, la lógica es:
1. Saca una muestra del dataset.
2. Entrena tu modelo con las $n-1$ muestras.
3. Evalúa tu modelo en la muestra que quedó fuera con el métrico que más se ajuste a la aplicación.
4. Regresa la muestra al dataset.
5. Repite 1-4 con muestras diferentes hasta haber hecho el procedimiento $n$ veces para $n$ muestras.
6. Calcula la media y desviación estándar de los métricos guardados.

Con los resultados del proceso de validación podemos saber qué tan bueno podría ser el modelo seleccionado con los datos (con/sin transformaciones).

### Ejercicio 1

Utiliza el dataset `Motor Trend Car Road Tests`. Elimina la columna `model` y entrena 32 modelos diferentes utilizando Leave-One-Out Cross Validation con target `mpg`. Utiliza MSE como métrico.

In [5]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import mean_squared_error
import numpy as np

In [2]:
#Carga de archivos
df = pd.read_excel("Motor Trend Car Road Tests.xlsx")

In [6]:
df.head()

,model,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
0,Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
1,Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
2,Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
3,Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
4,Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2


In [23]:
# Crear las matrices de datos de X (características) e y (variable objetivo)
X = df.drop(['model', 'mpg'], axis=1)
y = df['mpg']

# Ponmos lo necesario para el Leave-One-Out Cross-Validation
n = len(X)
mse_list = []

# Realizamos el Leave One-Out Cross-Validation manualmente
for i in range(n):
    
    X_train = X.drop(X.index[i])
    y_train = y.drop(y.index[i])
    
    X_test = X.iloc[[i]]
    y_test = y.iloc[[i]]
    
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    
    mse = mean_squared_error(y_test, y_pred)
    mse_list.append(mse)

# Calcula la media y desviación estándar de los MSE obtenidos
promedio_mse = np.mean(mse_list)
desviacion_mse = np.std(mse_list)

print(f"MSE Promedio: {promedio_mse}")
print(f"Desviación Estándar del MSE: {desviacion_mse}")
print(f"Numero de modelos entrenados: {len(mse_list)}")
print(f"La lista MSE: {mse_list}")

MSE Promedio: 12.181558006901955
Desviación Estándar del MSE: 17.06739987188854
Numero de modelos entrenados: 32
La lista MSE: [5.2588557966290645, 2.453996713086796, 20.55049341688269, 0.0443291071396446, 1.5811555159760267, 10.118625488875358, 0.016374041548473144, 8.08125071038954, 39.470183479649876, 0.7706327722901023, 4.955694234215864, 10.224730198354456, 4.430129331402683, 0.48781277124784195, 6.823844575809473, 0.6027506975546951, 36.81017860995872, 31.759855581210513, 1.063227541805789, 32.713286187027215, 14.307173682712689, 3.405354649998901, 9.408007649079883, 0.00010350039803653134, 9.962439534547606, 1.3412208919431396, 0.16480064082222703, 23.60020650224925, 83.1145764338518, 0.00010271237184881921, 8.785533367966966, 17.502929883865423]


Interpreta.

El MSE es alto (12) puesto que nuestra modelo de predicción en promedio es malo. 

La desviación estándar es tan alta (17) sugiere que el modelo es muy sensible a los datos de entrenamiento y que su capacidad de generalización varía drásticamente dependiendo del auto que se intenta predecir.

## K-Folds Cross-Validation

El dataset `Motor Trend Car Road Tests` sólo tiene 32 muestras, y utilizar un modelo sencillo de regresión múltiple hace que usar LOOCV sea muy rápido. El dataset `California Housing` tiene $20640$ muestras para $9$ columnas, entonces realizar un ajuste sobre una transformación o sobre el modelo y luego calcular el impacto esperado podría tomar más tiempo.

La solución propuesta es dividir el dataset en *k* folds (partes iguales), ajustar en *k-1* folds y probar en el restante.

### Ejercicio 2
Utiliza el dataset `California Housing` y haz K-folds Cross Validation con 10 folds. Utiliza el MSE como métrico.

In [ ]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
print("Dataset Shape:", housing.data.shape, housing.target.shape)
print("Dataset Features:", housing.feature_names)
print("Dataset Target:", housing.target_names)
X = housing.data
y = housing.target

Interpreta.

## Referencia

James, G., Witten, D., Hastie, T., Tibshirani, R.,, Taylor, J. (2023). An Introduction to Statistical Learning with Applications in Python. Cham: Springer. ISBN: 978-3-031-38746-3